## Game Description: Isolation
Isolation is a two-player, turn-based game played on a 7×7 board. Each player controls a single
piece and takes turns making moves. On each turn, a player must:
1. Move their piece like a queen in chess — in any direction (horizontally, vertically, or
diagonally) across any number of unblocked squares.
2. Block the square they just moved from. Once blocked, that square can no longer be
used by either player.

The game ends when one player is unable to make a legal move. In this case the other player is
declared the winner.

In [1]:
import copy

BOARD_SIZE = 7
EMPTY = 0
BLOCKED = -1
PLAYER1 = 1
PLAYER2 = 2

directions = [(-1, 0), (1, 0), (0, -1), (0, 1),
              (-1, -1), (-1, 1), (1, -1), (1, 1)]

def create_board():
    return [[EMPTY for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]

def print_board(board, p1_pos, p2_pos):
    display = [["." for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            if board[r][c] == BLOCKED:
                display[r][c] = "X"
    if p1_pos:
        display[p1_pos[0]][p1_pos[1]] = "1"
    if p2_pos:
        display[p2_pos[0]][p2_pos[1]] = "2"
    for row in display:
        print(" ".join(row))
    print()

def get_valid_moves(board, position, other_pos):
    moves = []
    r0, c0 = position
    for dr, dc in directions:
        r, c = r0 + dr, c0 + dc
        while 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE:
            if board[r][c] == BLOCKED or (r, c) == other_pos:
                break
            moves.append((r, c))
            r += dr
            c += dc
    return moves

def make_move(board, from_pos, to_pos):
    new_board = copy.deepcopy(board)
    fr, fc = from_pos
    new_board[fr][fc] = BLOCKED
    return new_board

def evaluate(board, player_pos, opponent_pos):
    player_moves = len(get_valid_moves(board, player_pos, opponent_pos))
    opponent_moves = len(get_valid_moves(board, opponent_pos, player_pos))
    return player_moves - opponent_moves

def alphabeta(board, depth, alpha, beta, maximizing, player_pos, opponent_pos):
    
    valid = get_valid_moves(board, player_pos, opponent_pos)
    
    if depth == 0 or not valid:
        return evaluate(board, player_pos, opponent_pos), None
    best_move = None

    if maximizing:
        value = float('-inf')
        for move in valid:
            new_board = make_move(board, player_pos, move)
            score, _ = alphabeta(new_board, depth - 1, alpha, beta, False, opponent_pos, move)
            if score > value:
                value, best_move = score, move
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return value, best_move
    
    else:
        value = float('inf')
        for move in valid:
            new_board = make_move(board, player_pos, move)
            score, _ = alphabeta(new_board, depth - 1, alpha, beta, True, opponent_pos, move)
            if score < value:
                value, best_move = score, move
            beta = min(beta, value)
            if alpha >= beta:
                break
        return value, best_move

def ai_move(board, player_pos, opponent_pos, depth=3):
    _, move = alphabeta(board, depth, float('-inf'), float('inf'), True, player_pos, opponent_pos)
    return move


def human_move(board, player_pos):
    return 0


def is_game_over(board, player_pos, other_pos):
    return not get_valid_moves(board, player_pos, other_pos)

def play_game(depth=3):
    board = create_board()
    p1_pos = (0, 0)
    p2_pos = (BOARD_SIZE - 1, BOARD_SIZE - 1)
    turn = PLAYER1
    print_board(board, p1_pos, p2_pos)
    while True:
        if turn == PLAYER1:
            valid_moves = get_valid_moves(board, p1_pos, p2_pos)
            print(f"Player 1 possible moves: {valid_moves}")
            move = ai_move(board, p1_pos, p2_pos, depth)
            print(f"AI (Player 1) moves to {move}")
            board = make_move(board, p1_pos, move)
            p1_pos = move
            if is_game_over(board, p2_pos, p1_pos):
                print_board(board, p1_pos, p2_pos)
                print("Player 1 wins!")
                break
            turn = PLAYER2
        else:
            valid_moves = get_valid_moves(board, p2_pos, p1_pos)
            print(f"Player 2 possible moves: {valid_moves}")
            move = ai_move(board, p2_pos, p1_pos, depth)
            print(f"AI (Player 2) moves to {move}")
            board = make_move(board, p2_pos, move)
            p2_pos = move
            if is_game_over(board, p1_pos, p2_pos):
                print_board(board, p1_pos, p2_pos)
                print("Player 2 wins!")
                break
            turn = PLAYER1
        print_board(board, p1_pos, p2_pos)
    print("Game over!")

if __name__ == "__main__":
    play_game(depth=3)


1 . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . 2

Player 1 possible moves: [(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]
AI (Player 1) moves to (1, 1)
X . . . . . .
. 1 . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . 2

Player 2 possible moves: [(5, 6), (4, 6), (3, 6), (2, 6), (1, 6), (0, 6), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0), (5, 5), (4, 4), (3, 3), (2, 2)]
AI (Player 2) moves to (1, 6)
X . . . . . .
. 1 . . . . 2
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . X

Player 1 possible moves: [(0, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (1, 0), (1, 2), (1, 3), (1, 4), (1, 5), (0, 2), (2, 0), (2, 2), (3, 3), (4, 4), (5, 5)]
AI (Player 1) moves to (2, 1)
X . . . . . .
. X . . . . 2
. 1 . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . X

Player 2 possible moves: [(0, 6), (2